In [1]:
import math
import pandas as pd
import re
import numpy as np
import itertools
import collections
from collections import defaultdict, Counter
import string
from sklearn.preprocessing import LabelEncoder

# Fulldata

In [2]:
ls

 Volume in drive C is Acer
 Volume Serial Number is 3818-7542

 Directory of C:\Users\phvpa\Desktop\Denoising_Experimants\ModifiedCode

14-11-2021  11:43    <DIR>          .
14-11-2021  11:38    <DIR>          ..
14-11-2021  11:37    <DIR>          .ipynb_checkpoints
01-11-2021  20:19    <DIR>          DataSets
30-10-2021  16:09            42,127 Kan_Preprocessing _and_spliting_2750and250.ipynb
14-11-2021  11:43            20,867 Mal_Preprocessing _and_spliting_250and50.ipynb
14-11-2021  11:36            43,263 Mal_Preprocessing _and_spliting_4500and500.ipynb
06-11-2021  11:14            44,233 Tamil_Preprocessing _and_spliting_1600and150.ipynb
               4 File(s)        150,490 bytes
               4 Dir(s)  379,389,767,680 bytes free


In [3]:
train_df = pd.read_csv("DataSets\Malyalam\Mal_sentiment_full_train.tsv", sep='\t')
print(len(train_df))

#test_df = pd.read_csv("Mal_sentiment_full_dev.tsv", header=None, sep='\t')
valid_df = pd.read_csv("DataSets\Malyalam\Mal_sentiment_full_dev.tsv", sep='\t')
print(len(valid_df))

test_df = pd.read_csv("DataSets\Malyalam\Mal_sentiment_full_test_withoutlabels.tsv", sep='\t')
len(test_df)

15888
1766


1962

In [4]:
# train_df

In [5]:
# train_df['category']

In [6]:
#removing all the emoji from the given data

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)


for i in range(len(train_df)):
    temp1 = deEmojify(train_df['text'][i])
    train_df['text'][i] = temp1

for j in range(len(valid_df)):
    temp2 = deEmojify(valid_df['text'][j])
    valid_df['text'][j] = temp2

for k in range(len(test_df)):
    temp3 = deEmojify(test_df['text'][k])
    test_df['text'][k]=temp3

In [7]:
def preprocess_data(text):
  user_name = '@[A-Za-z0-9]+' # Pattern for matching the user names in the tweet
  has_tag = '#[A-Za-z0-9]+' # Pattern for matching the hasgtags in the tweer
  url = 'https?:\/\/[A-Za-z0-9\.\/\-]+' # Pattern for matching the URLs in the tweet
  #emoji_pattern = r'/[x{1F600}-x{1F64F}]/u'
  
  x = text.lower()  # Lower casing all the characters
  x = re.sub(user_name, '', x) # Replace the username with an empty characrer
  x = re.sub(has_tag,'',x) # Replace the hashtags with an empty characrer
  x = re.sub(url, '', x) # Replace the URLs with an empty characrer
  #x = re.sub(emoji_pattern, '', x)
  x = x.translate(str.maketrans('', '', string.punctuation)) # Replace all the characters except alphabets and digits from the tweet

  return x

In [8]:
train_df['text'] = train_df['text'].apply(lambda x: preprocess_data(x)) # Apply the clean_data() to all the entries in the dataframe
valid_df['text'] = valid_df['text'].apply(lambda x: preprocess_data(x)) # Apply the clean_data() to all the entries in the dataframe
test_df['text'] = test_df['text'].apply(lambda x: preprocess_data(x)) # Apply the clean_data() to all the entries in the dataframe

In [9]:
Counter(train_df['category'])

Counter({'unknown_state': 5279,
         'Positive': 6421,
         'Negative': 2105,
         'Mixed_feelings': 926,
         'not-malayalam': 1157})

In [10]:
Counter(valid_df['category'])

Counter({'Positive': 706,
         'not-malayalam': 141,
         'unknown_state': 580,
         'Negative': 237,
         'Mixed_feelings': 102})

In [11]:
# np.save('DataSets\NumpyFiles\Mal\Fulldata\Mal_Train_Full_data.npy',list(train_df['text']))
# np.save('DataSets\NumpyFiles\Mal\Fulldata\Mal_Train_Full_labels.npy',list(train_df['category']))
# np.save('DataSets\NumpyFiles\Mal\Fulldata\Mal_Val_Full_data.npy',list(valid_df['text']))
# np.save('DataSets\NumpyFiles\Mal\Fulldata\Mal_Val_Full_labels.npy',list(valid_df['category']))
# np.save('DataSets\NumpyFiles\Mal\Fulldata\Mal_Test_Full_data.npy',list(valid_df['text']))

In [12]:
# temp1 = np.load('NumpyFiles\Mal\Fulldata\Mal_Train_Full_data.npy')
# temp2 = np.load('NumpyFiles\Mal\Fulldata\Mal_Train_Full_labels.npy')
# temp3 = np.load('NumpyFiles\Mal\Fulldata\Mal_Val_Full_data.npy')
# temp4 = np.load('NumpyFiles\Mal\Fulldata\Mal_Val_Full_labels.npy')
# temp5 = np.load('NumpyFiles\Mal\Fulldata\Mal_Test_Full_data.npy')

In [13]:
print(test_df['text'][279])
print(test_df['text'][516])
print(test_df['text'][546])
print(test_df['text'][555])
print(test_df['text'][559])
print(test_df['text'][570])
print(test_df['text'][825])
print(test_df['text'][838])
print(test_df['text'][860])
print(test_df['text'][890])
print(test_df['text'][1234])

oru panipalal manakunnu again mr jr 
ende ponno 68 vayasaya oraluda body thanneyano ith  namichu ponnikkaaaa
fdfs trailer kandu ithu vareyum njanithu paranjittilla
degraders step back   മമ്മൂക്ക
ഈ പാട്ട് ഡിസ്‌ലൈക് ചെയ്തവരുടെ വീട്ടുകാരെ സമ്മതിക്കണം big salute
ഇത് ഏത് ഭാഷ ശിവനെ ഏത് ജില്ല 
ഇതൊക്കെ മമ്മുക്ക പറഞ്ഞാൽ പൊളിക്കും   
108 st pius x kuttikanam        uyir
ente ponno heavy bgm ikka pinne parayanilla2 looks kidilosky  overall kidukkachi aahn mone
song dislike adicha 90 perum phycho singles akum
അഭ്യർത്ഥന  മാമാങ്കം പലകുറി കൊണ്ടാടും  എന്ന ഗാനം ടൈറ്റിൽ സോങ്ങ് ആയി ഉൾപ്പെടുത്താമോ  album vasantha geethangal singer k j yesudas


In [14]:
valid_df['text'][0]

'mammooka ninghal mass aa pwoli item'

# Dividing train and valid separatly:

### Val data

In [15]:
df_temp2 = valid_df.sort_values('category')
df_temp2

,text,category
974,tovinok pakaram unni mukundan mathiyarunnu,Mixed_feelings
166,puthanpanam movie pole thonniyath enikk maathr...,Mixed_feelings
536,richie orma vanathu eniku mathram ano,Mixed_feelings
700,friendship dayikk oru cake medichu kodukk fresh,Mixed_feelings
1363,pulimurugan copy adi pole ind stund seen ellam,Mixed_feelings
...,...,...
436,tesla code um കണ്ടിട്ട് ഇറങ്ങിയിട്ടുണ്ട് കുറേ,unknown_state
1180,കമന്റ്‌ വായിക്കുന്ന ലാലേട്ടൻ ഫാൻസിന് ലൈക് അടിക...,unknown_state
1182,eth sajiv pillai visuals ane alle,unknown_state
1186,katha manasilay tv il varambam kanam,unknown_state


In [16]:
val = Counter(valid_df['category'])
val_sort = dict(sorted(val.items(), key=lambda item: item[0]))

print('\nvaliddf:\n',val_sort,'\n')
temp = 0
for key in val_sort:
    print(key,'starts from: ', temp)
    temp = temp + int(val_sort[key])

print(key,'ends at: ', temp)


validdf:
 {'Mixed_feelings': 102, 'Negative': 237, 'Positive': 706, 'not-malayalam': 141, 'unknown_state': 580} 

Mixed_feelings starts from:  0
Negative starts from:  102
Positive starts from:  339
not-malayalam starts from:  1045
unknown_state starts from:  1186
unknown_state ends at:  1766


In [17]:
count2 = 10
val_Mixed_feelings = df_temp2[:count2]
val_Negative = df_temp2[110:110+count2]
val_Positive=df_temp2[350:350+count2]
val_not_malayalam = df_temp2[1060:1060+count2]
val_unknown_state = df_temp2[1190:1190+count2]

In [18]:
Val_frames = [val_Mixed_feelings, val_Negative, val_Positive, val_not_malayalam, val_unknown_state ]
Val_results = pd.concat(Val_frames)

In [19]:
Counter(Val_results['category'])

Counter({'Mixed_feelings': 10,
         'Negative': 10,
         'Positive': 10,
         'not-malayalam': 10,
         'unknown_state': 10})

In [20]:
Sub_val = Val_results.sample(frac = 1)

In [21]:
Sub_val_list = Sub_val['text'].tolist()
Sub_val_labels_list = Sub_val['category'].tolist()

### Train Data

In [22]:
train = Counter(train_df['category'])
train_sort = dict(sorted(train.items(), key=lambda item: item[0]))

print('Traindf: \n',train_sort,'\n')
temp = 0
for key in train_sort:
    print(key,'starts from: ', temp)
    temp = temp + int(train_sort[key])
print(key,'ends at: ', temp)

Traindf: 
 {'Mixed_feelings': 926, 'Negative': 2105, 'Positive': 6421, 'not-malayalam': 1157, 'unknown_state': 5279} 

Mixed_feelings starts from:  0
Negative starts from:  926
Positive starts from:  3031
not-malayalam starts from:  9452
unknown_state starts from:  10609
unknown_state ends at:  15888


In [23]:
df_temp = train_df.sort_values('category')
count = 50
Mixed_feelings = df_temp[:count]
Negative = df_temp[930:930+count]
Positive=df_temp[3040:3040+count]
not_malayalam = df_temp[9460:9460+count]
unknown_state = df_temp[10620:10620+count]

In [24]:
train_frames = [Mixed_feelings, Negative, Positive, not_malayalam, unknown_state ]

train_result = pd.concat(train_frames) #concat

In [25]:
Counter(train_result['category'])

Counter({'Mixed_feelings': 50,
         'Negative': 50,
         'Positive': 50,
         'not-malayalam': 50,
         'unknown_state': 50})

In [26]:
Sub_train = train_result.sample(frac = 1) #shuffle

In [27]:
Sub_train_list = Sub_train['text'].tolist()
Sub_labels_list = Sub_train['category'].tolist()

# Saving Choosen data into numpy files

In [29]:
Mal_val = np.array(Sub_val_list)
np.save("DataSets/NumpyFiles/Mal/Split250and50/Mal_Val_data_50.npy",Mal_val)
Mal_val_label = np.array(Sub_val_labels_list)
np.save("DataSets/NumpyFiles/Mal/Split250and50/Mal_val_label_50.npy",Mal_val_label)

In [30]:
temp11 =np.load('DataSets/NumpyFiles/Mal/Split250and50/Mal_Val_data_50.npy')
print(temp11.shape)
temp12 =np.load('DataSets/NumpyFiles/Mal/Split250and50/Mal_val_label_50.npy')
print(temp12.shape)

(50,)
(50,)


In [31]:
Mal_train = np.array(Sub_train_list)
np.save('DataSets/NumpyFiles/Mal/Split250and50/Mal_Train_data_250.npy',Mal_train)
Mal_train_label = np.array(Sub_labels_list)
np.save('DataSets/NumpyFiles/Mal/Split250and50/Mal_Train_label_250.npy',Mal_train_label)

In [32]:
temp21 =np.load('DataSets/NumpyFiles/Mal/Split250and50/Mal_Train_data_250.npy')
print(temp21.shape)
temp22 =np.load('DataSets/NumpyFiles/Mal/Split250and50/Mal_Train_label_250.npy')
print(temp22.shape)

(250,)
(250,)


In [33]:
Traindf = pd.DataFrame({'Data' : temp21,'label' : temp22}, columns=['Data', 'label'])
Traindf

,Data,label
0,ente chunkum ingane thanne ee video phonil kan...,unknown_state
1,മൂക്കുത്തി കണ്ടുപിടിക്കാൻ നമുക്കവരെ സഹായിക്കാം...,unknown_state
2,kannondu chollanu mindandu mindanu aaaa oru f...,Mixed_feelings
3,hukoo for rajinikanth sir hukku hu hu for moha...,not-malayalam
4,ijjathi trailer uff ponnooo oru raksha illaa p...,Positive
...,...,...
245,മാമാങ്കം എന്ന് കേൾക്കുമ്പോൾ ഞങ്ങൾ വള്ളുവനാട്ടു...,Negative
246,nammude ravi annante directionil oru padam kaa...,unknown_state
247,all kerala thalapathy bloods wishes mohan lal,unknown_state
248,ഇതിന്റെ തലക്ക് വല്ല ഓളവും ഉണ്ടോ പിന്നെ പാദസരം ...,Negative


In [34]:
Testdf = pd.DataFrame({'Data' : temp11,'label' : temp12}, columns=['Data', 'label'])
Testdf

,Data,label
0,ഇച്ചാപ്പി ഹസീബ് combo pinneyum kaaanan patyeel...,Positive
1,ampampo enna mathiri siripp ithu sirippille at...,not-malayalam
2,besttrailer adipoli ayal parayum ithu kanditu ...,Mixed_feelings
3,srk ടെ fan പോലെ ആവാതെ ഇരുന്നാ മതി,Mixed_feelings
4,oru mookuthhi kanathathinu dance kalikunnath i...,unknown_state
5,ithu oronnonnara allaaaa athukum mele mammook...,Positive
6,romanjification enta ponnikka nigal vere leval...,Positive
7,richie orma vanathu eniku mathram ano,Mixed_feelings
8,ആ നഴ്സറിക്ടാവിന്റെ മുമ്പിൽ വച്ച് ഇമ്മാതിരി കാബ...,unknown_state
9,bgm poaraaa goapi aanel maanthittaanelum bgm g...,Negative
